## <b> Carga de librerias </b>

In [ ]:
import pandas as pd 
import numpy as np 
import time
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# red neuronal
import keras 
from keras.models import Sequential
from keras.layers import Dense

# K-folds
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

# Grid Search 
from sklearn.model_selection import GridSearchCV

## <b> Carga de datos </b>
0 Standard,
1 Poor,
2 Good

In [ ]:
dataset = pd.read_csv(r'dataset procesados/modelado.txt', sep=';')
dict_cod = {'Standard':0, 'Poor':1, 'Good':2}
dataset['Credit_Score'] = dataset['Credit_Score'].map(dict_cod)
dataset.head()

## <b> Balance de datos </b>

In [ ]:
dataset['Credit_Score'].value_counts()

In [ ]:
nGood = len(dataset[dataset['Credit_Score']==2])
standard = dataset[dataset['Credit_Score']==0]
poor = dataset[dataset['Credit_Score']==1]
good = dataset[dataset['Credit_Score']==2]


standard = standard.sample(2*nGood)
dataset = pd.concat([standard, poor, good])
dataset = dataset.sample(frac=1)

In [ ]:
dataset['Credit_Score'].value_counts()

## <b> Verificación de Nulos </b>

In [ ]:
dataset.isnull().values.any()

## <b> Definición de datos para modelado </b>

### Variables

In [ ]:
X = dataset.iloc[:, dataset.columns != 'Credit_Score']
y = dataset.loc[:, 'Credit_Score']

In [ ]:
X_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.30, shuffle=True)

## <b> Red neuronal </b>

In [35]:
parametros = {
# Tamaño de la muestra
    'batch_size': [5,10], 

# Cantidad de epocas
    'nb_epoch': [100,500],

# Optimizador 
    'optimizer':['adam', 'SGD']
}

In [36]:
def red_grid_search_v1(optimizer):
    clasificador = Sequential()
    clasificador.add(Dense(input_dim=20, units=64, activation = 'relu', kernel_initializer='uniform') )
    clasificador.add(Dense(units=128, activation = 'relu', kernel_initializer='uniform') )
    clasificador.add(Dense(units=128, activation = 'relu', kernel_initializer='uniform') )
    clasificador.add(Dense(units=128, activation = 'relu', kernel_initializer='uniform') )
    clasificador.add(Dense(units=3, activation='relu', kernel_initializer='uniform'))
    clasificador.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return clasificador

In [37]:
clasificador_grid_search = KerasClassifier(build_fn=red_grid_search_v1)

<ipython-input-37-f1420de21d75>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  clasificador_grid_search = KerasClassifier(build_fn=red_grid_search_v1)


In [38]:
grid_out = GridSearchCV(estimator=clasificador_grid_search,
                        param_grid=parametros,
                        cv=20,
                        scoring='accuracy')

In [39]:
train_gs_out = grid_out.fit(X_train, y_train, verbose=1)

5769/5769 [==============================] - 26s 4ms/step - loss: 6.2356 - accuracy: 0.4336


In [40]:
grid_out.best_params_

{'batch_size': 10, 'nb_epoch': 500, 'optimizer': 'SGD'}

In [41]:
grid_out.best_score_

0.35729002660948955

In [43]:
from sklearn.metrics import classification_report
print(classification_report(y_train, grid_out.predict(X_train) ) )

1803/1803 [==============================] - 5s 3ms/step
              precision    recall  f1-score   support

           0       0.43      1.00      0.60     25012
           1       0.00      0.00      0.00     20221
           2       0.00      0.00      0.00     12451

    accuracy                           0.43     57684
   macro avg       0.14      0.33      0.20     57684
weighted avg       0.19      0.43      0.26     57684



c:\Users\apatz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\apatz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\apatz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
